In [1]:
import pandas as pd

# -------------------------------
# 📁 파일 불러오기 (경로 수정 필요)
# -------------------------------
file_path = r"C:\Users\owner\Desktop\LG Aimers\open\test\TEST_00.csv"
df = pd.read_csv(file_path)

# 날짜 변환 및 요일 추출
df['영업일자'] = pd.to_datetime(df['영업일자'])
df['요일번호'] = df['영업일자'].dt.dayofweek
df['요일'] = df['요일번호'].map({
    0: '월', 1: '화', 2: '수', 3: '목', 4: '금', 5: '토', 6: '일'
})

# 영업장, 메뉴 분리
df[['영업장명', '메뉴명']] = df['영업장명_메뉴명'].str.split('_', expand=True)
df.drop(columns=['영업장명_메뉴명'], inplace=True)

# ---------------------------------------------------------------
# 🧱 날짜 보간 (누락 날짜 채우기 위해 full index 생성)
# ---------------------------------------------------------------
all_dates = pd.date_range(df['영업일자'].min(), df['영업일자'].max())
menu_keys = df[['영업장명', '메뉴명']].drop_duplicates()
menu_keys['key'] = 1
full_dates = pd.DataFrame({'영업일자': all_dates, 'key': 1})
full_df = pd.merge(menu_keys, full_dates, on='key').drop(columns=['key'])

# 원본 데이터 병합 (매출수량 누락 시 NaN 발생)
df = pd.merge(full_df, df, on=['영업일자', '영업장명', '메뉴명'], how='left')

# 다시 요일 생성
df['요일번호'] = df['영업일자'].dt.dayofweek
df['요일'] = df['요일번호'].map({
    0: '월', 1: '화', 2: '수', 3: '목', 4: '금', 5: '토', 6: '일'
})

# 매출수량 NaN → 0
df['매출수량'] = df['매출수량'].fillna(0)

# ---------------------------------------------------------------
# 🚩 공휴일 컬럼 추가 (대한민국 2023년 공휴일 기준)
# ---------------------------------------------------------------
holiday_list = [
    '2023-01-01', '2023-01-21', '2023-01-22', '2023-01-23',
    '2023-03-01', '2023-05-05', '2023-05-27', '2023-06-06',
    '2023-08-15', '2023-09-28', '2023-09-29', '2023-09-30',
    '2023-10-03', '2023-10-09', '2023-12-25'
]
holiday_dates = pd.to_datetime(holiday_list)
df['공휴일여부'] = df['영업일자'].isin(holiday_dates).astype(int)

# ---------------------------------------------------------------
# 💾 저장
# ---------------------------------------------------------------
save_path = r"C:\Users\owner\Desktop\LG Aimers\open\test\aaTEST_00.csv"
df.to_csv(save_path, index=False, encoding='utf-8-sig')

print(f"저장 완료: {save_path}")


저장 완료: C:\Users\owner\Desktop\LG Aimers\open\test\aaTEST_00.csv
